In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks

# 1. Setting Folder (Pastiin folder 'dataset' lu ada di lokasi yang bener)
train_dir = r'C:\Users\ADAM FIRDAUS\Downloads\dev_pneumonia-main\dataset\train' 

# 2. Data Augmentation (Rahasia biar akurasi gak ngaco lagi)
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2 # Otomatis bagi 20% data buat ujian si AI
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# 3. Arsitektur Model (Versi lebih kuat buat 1000 data)
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5), # Biar AI gak cuma ngafalin posisi pixel
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Early Stopping (Biar kalau udah pinter gak perlu nunggu sampe 30 epoch)
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 5. GAS Training!
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[stop_early]
)

# 6. Simpan Hasilnya (Biarkan ini jalan sampe beres)
model.save('model_pneumonia_v2_1000data.h5')
print("ALHAMDULILLAH! Model baru yang lebih pinter udah kesimpen!")

In [ ]:
# Kodingan Penyelamat agar Visualisasi tidak Error
val_generator.shuffle = False
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()
y_true = val_generator.classes
# Ambil angka matrix buat visualisasi nanti
conf_matrix = confusion_matrix(y_true, y_pred_classes)

In [ ]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Ubah data_flow jadi train_generator
report = classification_report(y_true, y_pred_classes, target_names=list(train_generator.class_indices.keys()))
print("\nClassification Report:")
print(report)

In [ ]:
# Visualisasi akurasi dan loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Ubah data_flow jadi train_generator
report = classification_report(y_true, y_pred_classes, target_names=list(train_generator.class_indices.keys()))
print("\nClassification Report:")
print(report)

In [ ]:
# Menampilkan metrik
from sklearn.metrics import f1_score, recall_score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {np.mean(y_true == y_pred_classes) * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
# Ubah data_flow jadi train_generator
report = classification_report(y_true, y_pred_classes, target_names=list(train_generator.class_indices.keys()))
print("\nClassification Report:")
print(report)

In [ ]:
print("Confusion Matrix:")
print(conf_matrix)
